In [1]:
%pylab inline
#import homer
import pandas as pd
import numpy as np
import dask.bag as db
from dask.diagnostics import ProgressBar
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
import json
from dask.diagnostics import visualize
import dask
import pickle
from itertools import combinations
import re
import dateutil.parser
from nltk.tokenize import RegexpTokenizer
import ujson
import dask.dataframe as dd
import itertools
from dask import delayed
import collections
from bokeh.io import output_notebook
output_notebook()

Populating the interactive namespace from numpy and matplotlib


Loading BokehJS ...

In [15]:
import time
str(time.time()).replace('.', '_')

'1486670904_929779'

In [31]:
dates = [20150616, 20150617, 20150618, 20150619, 20150620, 20150621, 20150622]
tw_files='/Users/houghton/Desktop/tw/posts_sample_201506*.gz'

In [2]:
dates = [20150615]
thresholds = [1]
tw_files='tests/resources/small_tw_file*.gz'

In [32]:
dates = [20150615, 20150616, 20150617, 20150618, 20150619, 20150620, 
         20150621, 20150622, 20150623, 20150624, 20150625]

sw_file = "homer/stopwords.pickle"
tw_stopwords = pickle.load(open(sw_file, "rb"))

tokenizer = RegexpTokenizer(r'\w+')
matcher = re.compile(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*')

languages = ['en']
def fold_pairs(counter, json_string):
    parsed_json = ujson.loads(json_string)
    if ('lang' in parsed_json and
            'text' in parsed_json and
            parsed_json['lang'] in languages):
        text = parsed_json['text'].lower()

        # remove hyperlinks
        text = matcher.sub('', text)

        # tokenize, dropping punctuation
        tokens = tokenizer.tokenize(text)  # this should release GIL - be ok for parallelization

        # drop stopwords
        tokens = filter(lambda x: x not in tw_stopwords, tokens)

        date = int(dateutil.parser.parse(parsed_json['created_at']).strftime("%Y%m%d"))
        
        if date not in counter:
            counter[date] = collections.Counter()
        for pair in combinations(tokens, 2):
            counter[date][tuple(sorted(pair))] += 1

    return counter


def name_counter():
    for i in range(1000):
        yield i
        
nc = name_counter()

def merge_folds(a, b):
    if isinstance(a, dict):
        for date, c in a.items():
            if len(c):
                pd.Series(c).to_csv('/Users/houghton/Desktop/tw/charleston/wel_%i_%s.csv'%(
                        date, str(time.time()).replace('.', '_')+'a'))

    if isinstance(b, dict):
        for date, c in b.items():
            if len(c):
                pd.Series(c).to_csv('/Users/houghton/Desktop/tw/charleston/wel_%i_%s.csv'%(
                        date, str(time.time()).replace('.', '_')+'b'))

    return None

In [33]:
lines = db.read_text(tw_files, compression='gzip', collection=True)

initial_counter = {date:collections.Counter() for date in dates}
counters = lines.fold(binop=fold_pairs, combine=merge_folds, initial=initial_counter)
        

In [34]:
with Profiler() as prof, ResourceProfiler(dt=5) as rprof, CacheProfiler() as cprof:
    counts = counters.compute()

visualize([prof, rprof, cprof])  

Exception ignored in: <bound method ResourceProfiler.close of <dask.diagnostics.profile.ResourceProfiler object at 0x1199337b8>>
Traceback (most recent call last):
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/site-packages/dask/diagnostics/profile.py", line 173, in close
    self._tracker.shutdown()
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/site-packages/dask/diagnostics/profile.py", line 210, in shutdown
    self.join()
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/multiprocessing/process.py", line 119, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method ResourceProfiler.close of <dask.diagnostics.profile.ResourceProfiler object at 0x119270668>>
Traceback (most recent call last):
  File "/Users/houghton/anaconda/envs/py3k/lib/python3.5/site-packages/dask/diagnostics/profile.py", line 173, in close
    self._tracker.shutdown()
  File "/

Column(id='0c6f5fdd-5f2f-46eb-a7ef-36faee7c7db2', ...)